# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


#### Usage:

In [1]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

Using TensorFlow backend.


17473536/17464789 [==============================] - 1s 0us/step


In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [0]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [0]:
#Importing Libraries

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding,LSTM
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
# Make Word to ID dictionary
INDEX_FROM=3   # word index offset
word_to_id = imdb.get_word_index() #Get the word index
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}

word_to_id["<PAD>"] = 0

word_to_id["<START>"] = 1

word_to_id["<UNK>"] = 2

# Make ID to Word dictionary
id_to_word = {value:key for key,value in word_to_id.items()}

1654784/1641221 [==============================] - 1s 0us/step


In [6]:
id_to_word

{0: '<PAD>',
 1: '<START>',
 2: '<UNK>',
 4: u'the',
 5: u'and',
 6: u'a',
 7: u'of',
 8: u'to',
 9: u'is',
 10: u'br',
 11: u'in',
 12: u'it',
 13: u'i',
 14: u'this',
 15: u'that',
 16: u'was',
 17: u'as',
 18: u'for',
 19: u'with',
 20: u'movie',
 21: u'but',
 22: u'film',
 23: u'on',
 24: u'not',
 25: u'you',
 26: u'are',
 27: u'his',
 28: u'have',
 29: u'he',
 30: u'be',
 31: u'one',
 32: u'all',
 33: u'at',
 34: u'by',
 35: u'an',
 36: u'they',
 37: u'who',
 38: u'so',
 39: u'from',
 40: u'like',
 41: u'her',
 42: u'or',
 43: u'just',
 44: u'about',
 45: u"it's",
 46: u'out',
 47: u'has',
 48: u'if',
 49: u'some',
 50: u'there',
 51: u'what',
 52: u'good',
 53: u'more',
 54: u'when',
 55: u'very',
 56: u'up',
 57: u'no',
 58: u'time',
 59: u'she',
 60: u'even',
 61: u'my',
 62: u'would',
 63: u'which',
 64: u'only',
 65: u'story',
 66: u'really',
 67: u'see',
 68: u'their',
 69: u'had',
 70: u'can',
 71: u'were',
 72: u'me',
 73: u'well',
 74: u'than',
 75: u'we',
 76: u'much',
 

In [7]:
print("Train-set size: ", len(x_train))
print("Test-set size:  ", len(x_test))

('Train-set size: ', 25000)
('Test-set size:  ', 25000)


In [8]:
np.array(x_train[1])

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    1,  194, 1153,  194, 8255,   78,  228,    5,    6, 1463,
       4369, 5012,  134,   26,    4,  715,    8,  118, 1634,   14,  394,
         20,   13,  119,  954,  189,  102,    5,  207,  110, 3103,   21,
         14,   69,  188,    8,   30,   23,    7,   

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [0]:
model = Sequential()
embedding_size = 8

In [0]:
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    name='Embedding_layer'))

In [0]:
model.add(LSTM(units=16, return_sequences=True))

model.add(LSTM(units=8, return_sequences=True))

model.add(LSTM(units=4))

model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_layer (Embedding)  (None, None, 8)           80000     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 16)          1600      
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 8)           800       
_________________________________________________________________
lstm_3 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 82,613
Trainable params: 82,613
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(x_train, y_train,
          validation_split=0.05, epochs=2, batch_size=64)

/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 23750 samples, validate on 1250 samples
Epoch 1/2
23750/23750 [==============================] - 389s 16ms/step - loss: 0.4731 - accuracy: 0.7781 - val_loss: 0.4597 - val_accuracy: 0.8168
Epoch 2/2
23750/23750 [==============================] - 390s 16ms/step - loss: 0.2716 - accuracy: 0.9008 - val_loss: 0.3258 - val_accuracy: 0.8680


In [15]:
result = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 134s 5ms/step


In [16]:
print("Accuracy: {}".format(result[1]))

Accuracy: 0.859839975834


In [17]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 85.98%


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [0]:
import tensorflow as tf
from keras import backend as k

In [24]:
#Define function to get output of all the layers in the model for specific test input
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

    def getLayerOutput(layer):
        get_Layer_Output = k.function([model.layers[0].input], [layer.output])
        return get_Layer_Output([x_test[0:1,]])[0]
    
    layer_output = []
    
    for layer in model.layers:
        layer_output.append(getLayerOutput(layer))

print(layer_output)

AttributeError: ignored

In [25]:
# Getting a specific output layer
layer_output[2]

NameError: ignored